In [1]:
import numpy as np
import os
import subprocess 
import MDAnalysis as md
from MDAnalysis.analysis.leaflet import LeafletFinder
import re

# Notebook for constructing a Bleb LNP with 1000nt Poly A RNA

## Requirements
### - Define Paths bellow;
### - Install MDAnalysis, Packmol, TS2CG, mdvcontainment, freud and mdvwhole.

In [2]:
IL_name_NP = 'MC3H'
IL_name_N1 = 'MC3'
sterol_name = 'CHOL'
noRNA = 1
vesicle_radius = 24 # nm

In [3]:
main_dir = '.'
mdp_loc_BL = 'MDPs/'
ILs_itp = 'ITP/MC3_KC2_DP_DT/_LI5_LI2_LI10_BMHB.itp'

PCG_path = '' #Define path to PCG/TS2CG instalation

RNA = 'PDBs/1000_RNA.pdb'
RNA_itp = 'ITP/1000_RNA_10-to-2.itp'

    
itpDir   = "ITP/"
IL_itp   = 'ITP/MC3_KC2_DP_DT_LI5_LI2_LI10_BMHB.itp'
Sterols  = "ITP/martini_v3.0_sterols_v1.0.itp"    
ffbonded = "ITP/martini_v3.0_ffbonded.itp"

####  functions

In [4]:
def get_radius (x,y,z):
    # Calculate the centroid of the cylinder (center axis)
    centroid_x = np.mean(x)
    centroid_y = np.mean(y)
    centroid_z = np.mean(z)

    # Calculate the distances from the centroid to each point
    distances = np.sqrt((x - centroid_x)**2 + (y - centroid_y)**2 + (z - centroid_z)**2)

    # Find the minimum distance, which represents the internal radius of the cylinder
    internal_radius = np.min(distances)

    return internal_radius

def write_packmol_input(channel_radius, noW=1, noCL=1, channel_length=100):
    f = open('solvate_channel.inp', 'w') 
    f.write(f'tolerance 3.0\nfiletype pdb\n\n')

    f.write(f'structure {RNA}\n')
    f.write(f'  number {noRNA}\n')
    f.write(f'  inside sphere 0. 0. 0. {(int(channel_radius))}.\n')
    f.write(f'end structure\n')

    f.write(f'structure PDBs/water.pdb\n')
    f.write(f'  number {noW}\n')
    f.write(f'  inside sphere 0. 0. 0. {int(channel_radius)}.\n')
    f.write(f'end structure\n')

    f.write(f'structure PDBs/CL.pdb\n')
    f.write(f'  number {noCL}\n')
    f.write(f'  inside sphere 0. 0. 0. {int(channel_radius)}.\n')
    f.write(f'end structure\n')
            
    f.write('output solvate_channel_rna.pdb\n')
    f.close()
    return

def extract_lines(input_file, pattern):
    extracted_lines = []
    pattern_found = False

    with open(input_file, 'r') as f:
        for line in f:
            if pattern_found:
                extracted_lines.append(line)

            if pattern in line:
                pattern_found = True

    return extracted_lines

def write_topol (topol, noW, noCL):
    '''Writes out a topol.top file with the solvate channel times 4 including the correct itp files'''
    extracted_lines = extract_lines(topol, 'lower monolayer')
    
    with open('topol.top', 'w') as topout:
        topout.write(f'#include "{itpDir}/martini_v3.0.0.itp"\n#include "{ffbonded}"\n#include "{RNA_itp}"\n\
#include "{itpDir}/martini_v3.0.0_phospholipids_v1.itp"\
\n#include "{IL_itp}"\n#include "{Sterols}"\n#include "{itpDir}/martini_v3.0.0_solvents_v1.itp"\n\
#include "{itpDir}/martini_v3.0.0_ions_v1.itp"\n')
        topout.write(f'[ system ]\n')
        topout.write('Compartment_RNA\n')
        topout.write(f'\n')
        topout.write(f'[ molecules ]\n')
        for i in range(1):
                for line in extracted_lines:
                    topout.write(line)
                topout.write(f'RNA    {noRNA}\n')
                topout.write(f'W    {noW}\n')
                topout.write(f'CL    {noCL}\n')
    return

##### write input file for pcg

In [412]:
O = open('input.str', 'w')

O.write("[Lipids List]\n")
O.write("Domain 0\n")
O.write(f"{IL_name_NP} 0.67  0.67 0.60\n") 
O.write(f"{sterol_name} 0.33  0.33 0.60\n") 
O.write("End\n")
O.write("\n")
O.write("[Shape Data]\n")
O.write("ShapeType Sphere\n")
O.write(f"Box {vesicle_radius}*2 {vesicle_radius}*2 {vesicle_radius}*2 \n")
O.write("Density 2\n")
O.write("WallDensity 1 1\n")
O.write("Thickness 3.8\n")
O.write("DL 0.2\n")
O.write(f"Radius {vesicle_radius} \n")
O.write("End\n")
O.close()

##### Build the sphere with PCG

In [413]:
p = subprocess.Popen(f"{PCG_path} -str input.str -Bondlength 0.2 -LLIB Martini3.LIB -defout system -function analytical_shape"
                    , stdin=subprocess.PIPE, shell=True, universal_newlines=True)
p.wait()

==============  PCG membrane builder: from function to membrane ======================
****************** CG Membrane builder ******************   
******************  Version:  1.1 ****************** 
[Lipids List]
Domain 0
MC3H 0.67  0.67 0.60
CHOL 0.33  0.33 0.60
End

[Shape Data]
ShapeType Sphere
vesicle will be made 
--> Generating molecule types from  input.str  file
--> Note: the lipids will be generated from < Martini Map CG> Lipid Library, of version:  Martini 3
--> This library contains 146 lipid types 
--> Note: the total upper monolayer area is 8429.65 and the total lower monolayer area is 6137.54
 now is time to add the lipids 
 Number of the domains defined in the input file  1
***************************  we aim to generate  ********************** 
*     For domain with ID 0 
*     9413  MC3H     
*     4636  CHOL     
   in the upper monolayer 
*     For domain with ID 0 
*     6853  MC3H     
*     3375  CHOL     
   in the lower monolayer 
*************************** 

0

##### Extract the inner leaflet

In [414]:
u = md.Universe('system.gro')

L = LeafletFinder(u, 'name NP N1 ROH', cutoff=15)

inner_res = ' '.join(f'{i}' for i in L.groups()[1].resids)
inner = u.select_atoms(f'resid {inner_res}')
inner.atoms.write('Inner.pdb')

##### solvate and add neutralizing ions using packmol

In [6]:
u = md.Universe('Inner.pdb')
sel = u.select_atoms('name N1 NP ROH')
sel_lipid = u.select_atoms('name N1 NP')
R = get_radius(sel.positions[:,0], sel.positions[:,1], sel.positions[:,2]) - 1  #radius of the channel, trying to substract 2 angstorm to avoid clashing

In [7]:
u.atoms.positions = u.atoms.positions - u.atoms.center_of_mass()
u.atoms.write('Inner_centered.pdb')

/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'elements' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


In [8]:
u = md.Universe('Inner_centered.pdb')
sel = u.select_atoms('name N1 NP ROH')
bb = u.select_atoms('name BB')

In [9]:
hydration = 3 #This parameter this to be fixed iteratively
noL = sel.resids.shape[0]
noCL = sel_lipid.resids.shape[0] - bb.resids.shape[0]
noW = (noL*hydration) - noCL

In [10]:
noW

23831

In [11]:
write_packmol_input(R, noW=noW, noCL=noCL)

##### Run packmol - Only use packmol for solvate and add ions

In [12]:
!/projects/DAMM/packmol-20.14.3/packmol < solvate_channel.inp


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                              Version 20.14.3 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Types of coordinate files specified: pdb
  Seed for random number generator:      1234567
  Output file: solvate_channel_rna.pdb
  Reading coordinate file: PDBs/1000_RNA.pdb
  Reading coordinate file: PDBs/water.pdb
  Reading coordinate file: PDBs/CL.pdb
  Number of independent structures:            3
  The structures are: 
  Structure            1 :PDBs/1000_RNA.pdb(       12000  atoms)
  Structure            2 :PDBs/water.pdb(           1  atoms)
  Structure            

In [13]:
## Join membrane and Water core
u1 = md.Universe("Inner_centered.pdb")
u2 = md.Universe("solvate_channel_rna.pdb")

u = md.Merge(u1.atoms, u2.atoms)
u.atoms.write("combined.pdb")

/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/PDBParser.py:348: UserWarning: Unknown element  found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: 
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/DAMM/hosts/damm085/data1/mvalerio/my_env/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "


##### Create box

In [14]:
os.system('gmx editconf -f combined.pdb -o combined.gro -box 70 70 70') 

                     :-) GROMACS - gmx editconf, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/LNP_builder/Building_material/Bleb_particle
Command line:
  gmx editconf -f combined.pdb -o combined.gro -box 70 70 70


Back Off! I just backed up combined.gro to ./#combined.gro.1#


Note that major changes are planned in future for editconf, to improve usability and utility.
Read 155295 atoms
Volume: 0.001 nm^3, corresponds to roughly 0 electrons
No velocities found
    system size : 46.383 46.188 46.176 (nm)
    center      : -0.011  0.066 -0.278 (nm)
    box vectors :  0.100  0.100  0.100 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       : 35.011 34.934 35.278 (nm)
new center      : 35.000 35.000 35.000 (nm)
new box vectors : 70.000 70.000 70.000 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :343000.00               (nm^3)



GROMACS reminds you: "You got one part of that wrong. This is not meth." (Breaking Bad)



0

##### Write topol.top

In [15]:
write_topol('system.top', noW, noCL)

### Min and relax

In [16]:
os.system(f"gmx grompp -f {mdp_loc_BL}/min_scp.mdp -c combined.gro -p topol.top -o m.tpr -maxwarn 1")

                      :-) GROMACS - gmx grompp, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/LNP_builder/Building_material/Bleb_particle
Command line:
  gmx grompp -f MDPs//min_scp.mdp -c combined.gro -p topol.top -o m.tpr -maxwarn 1


NOTE 1 [file MDPs//min_scp.mdp, line 43]:
  For proper sampling of the (nearly) decoupled state, stochastic dynamics
  should be used


NOTE 2 [file topol.top, line 17]:
  System has non-zero total charge: -1000.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.





Setting the LD random seed to -571739260

Generated 1689 of the 357435 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'RNA'

Excluding 1 bonded neighbours molecule type 'W'

Excluding 1 bonded neighbours molecule type 'CL'

Coupling 40913 copies of molecule type 'system'

Cleaning up constraints and constant bonded interactions with virtual sites

Cleaning up constraints and constant bonded interactions with virtual sites
Analysing residue names:
There are: 40912      Other residues
There are:  1000        RNA residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

This run will generate roughly 12 Mb of data


Number of degrees of freedom in T-Coupling group rest is 385132.00
The integrator does not provide a ensemble temperature, there is no system ensemble temperature

NOTE 3 [file MDPs//min_scp.mdp]:
  You are using a plain Coulomb cut-off, which might produce artifacts.
  You might want to consider using PME electrostatics.



There were 3 NOTEs

Back Off! I just backed up m.tpr to ./#m.tpr.1#

GROMACS reminds you: "I came up with the new convergence method, it's called a deadline driven convergence. My simulation is converged when it hits the deadline." (Tanadet Pipatpolkai)



0

In [ ]:
os.system('gmx mdrun -v -deffnm m -ntomp 5 -ntmpi 2')

In [19]:
### Create the index file
gro = 'm.gro'
u = md.Universe(gro)
lipidsagg=u.select_atoms(f'resname {IL_name_N1} {IL_name_NP} MC3H LI2H SITO CHOL A')
solventagg=u.select_atoms(f'not resname {sterol_name} LI2H MC3H SITO CHOL A')
lipidsagg.write("index.ndx", mode="w", name= 'Particle')
solventagg.write("index.ndx", mode="a", name= 'Solvent')
u.atoms.write("index.ndx", mode="a", name= 'System')

## Make whole and wrap mono- or bilayer around with PEGylated lipids

In [ ]:
os.system("mdvwhole -f m.tpr -x r.gro -o r_whole.gro -wa True -mol True")